In [1]:
# Import libraries and packages for the project 

from selenium import webdriver
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import random
import numpy as np
import pandas as pd
from underthesea import sentiment


print('- Finish importing packages')

- Finish importing packages


In [2]:
raw = pd.read_csv('ShopeeMall.csv')
raw.head()

,Order,Name,URL
0,1,Abbott,https://shopee.vn/abbottofficial
1,2,Adidas Official Store,https://shopee.vn/adidasofficialstore
2,3,Alobuy,https://shopee.vn/1khovn_bach_hoa_tien_loi
3,4,Amak,https://shopee.vn/nhasachamak
4,5,An Nguyên,https://shopee.vn/sachhayquanhieu


In [3]:
df = np.array(raw)

In [4]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver,20)
url = "https://shopee.vn/"

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [/home/dijkstra/.wdm/drivers/chromedriver/linux64/104.0.5112.20/chromedriver] found in cache


In [5]:

headers = ['Order', 'Name', 'URL', "Sales"]
update_data = []
char_dic = {'k':1000,'r': 1000000}

count = 0
sales = 0

for shop in tqdm(df,desc="Processing: "):
    product_list_url = []
    count = 0
    total = 0
    try:
        my_url = shop[2]+"?page=0&sortBy=sales"
        driver.get(my_url)
        print('- Accessing shop: ', shop[2])
        sleep(2)
        
    except:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue

    print('-- Getting data....')

    for i in range(3):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(0.3)
        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        sleep(0.3)
        old_page_source = driver.page_source

        items = loop_page_source.find('div',class_="shop-search-result-view").find_all('a',href=True)

        product_list_url.append(items)
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click()
        sleep(0.4)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            break
        sleep(2)
    print('-- Finished get data')

    print("--- Access product pages")
    print("************************")
    for page in product_list_url:
        if(count==101):
            break
        for products in page:
            count+=1
            if(count==101):
                break
            price = products.find('span',class_="_3TJGx5").text
            amount_raw = products.find('div',class_="_2R-Crv")

            price = price.replace(".","")

            if(not amount_raw):
                continue
            elif (len(amount_raw.text)==0):
                continue

            tmp = amount = amount_raw.text.split(" ")[2].split("/")[0].replace(",",".")

            if tmp[-1].isdecimal():
                amount = float(tmp)
            else:
                if(tmp[-1]=="k"):
                    amount = float(tmp[:-1])*char_dic[tmp[-1]]
                else:
                    amount = float(tmp[:-2])*char_dic[tmp[-1]]


            total += float(price)* amount

    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:total})
    print("************************")

print("Finished crawling data!")


Processing:   0%|          | 0/100 [00:00<?, ?it/s]

- Accessing shop:  https://shopee.vn/abbottofficial
-- Getting data....
-- Finished get data
--- Access product pages
************************
************************
- Accessing shop:  https://shopee.vn/adidasofficialstore
-- Getting data....
-- Finished get data
--- Access product pages
************************
************************
- Accessing shop:  https://shopee.vn/1khovn_bach_hoa_tien_loi
-- Getting data....
-- Finished get data
--- Access product pages
************************
************************
- Accessing shop:  https://shopee.vn/nhasachamak
-- Getting data....
-- Finished get data
--- Access product pages
************************
************************
- Accessing shop:  https://shopee.vn/sachhayquanhieu
-- Getting data....
-- Finished get data
--- Access product pages
************************
************************
- Accessing shop:  https://shopee.vn/anbertmangashop
-- Getting data....
-- Finished get data
--- Access product pages
************************
***

In [6]:
with open('ShopeeMallSales.csv', 'w',  newline = '') as file_output:
    
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')

Mission Completed!
